In [1]:
import pandas as pd
import json

input_ = pd.read_excel(f'./3.item.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Json_Compatibility'] != json.dumps('')].reset_index(drop=True)
input_['No'] = input_['No'].astype(int)

input_

,No,Item Number,Title,Subtitle,Price,Available,Sold,Vehicle W/o Engine,Vehicle W/ Engine,Picture,Url,Description Url,Json_Compatibility,Json_Src,Json_Specific
0,1,192511998212,Genuine Toyota LandCruiser UZJ100 HDJ101 HDJ10...,,AU $575.48 each,3 available,8 sold,,,,https://www.ebay.com.au/itm/192511998212?_ul=A...,https://vi.vipr.ebaydesc.com/itmdesc/192511998...,"{""market"": ""AU"", ""page"": 1, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/CZgAAOSw...","{""0"": {""Condition"": ""Brand New: A brand-new, u..."
1,2,195660971887,Genuine Toyota LandCruiser 100 Series UZJ HDJ ...,,AU $575.48 each,3 available,,,,,https://www.ebay.com.au/itm/195660971887?_ul=A...,https://vi.vipr.ebaydesc.com/itmdesc/195660971...,"{""market"": ""AU"", ""page"": 1, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/m0MAAOSw...","{""0"": {""Condition"": ""Brand New: A brand-new, u..."
2,3,192557565418,Genuine Lexus LX470 UZJ100 Heater Fan Blower M...,,AU $575.48 each,3 available,1 sold,,,,https://www.ebay.com.au/itm/192557565418?_ul=A...,https://vi.vipr.ebaydesc.com/itmdesc/192557565...,"{""market"": ""AU"", ""page"": 1, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/slgAAOSw...","{""0"": {""Condition"": ""Brand New: A brand-new, u..."
3,4,192335808616,Genuine Toyota LandCruiser VDJ200 UZJ200 URJ20...,,AU $426.61 each,10 available,110 sold,,,,https://www.ebay.com.au/itm/192335808616?_ul=A...,https://vi.vipr.ebaydesc.com/itmdesc/192335808...,"{""market"": ""AU"", ""page"": 2, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/8ysAAOSw...","{""0"": {""Condition"": ""Brand New: A brand-new, u..."
4,5,192470815384,Genuine Toyota LandCruiser 200 Series VDJ UZJ ...,,AU $426.61 each,10 available,4 sold,,,,https://www.ebay.com.au/itm/192470815384?_ul=A...,https://vi.vipr.ebaydesc.com/itmdesc/192470815...,"{""market"": ""AU"", ""page"": 2, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/TrUAAOSw...","{""0"": {""Condition"": ""Brand New: A brand-new, u..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,362,194505420306,Genuine Yaris NCP91 NCP151 NCP150 NCP131 Heate...,,AU $44.90 each,10 available,1 sold,,,,https://www.ebay.com.au/itm/194505420306?_ul=A...,https://vi.vipr.ebaydesc.com/itmdesc/194505420...,"{""market"": ""AU"", ""page"": 11, ""data"": {""scopedC...","{""0"": ""https://i.ebayimg.com/images/g/XGcAAOSw...","{""0"": {""Condition"": ""Brand New: A brand-new, u..."
334,363,194505420468,Genuine Yaris NSP131 NSP130 NLP90 NLP130 Heate...,,AU $44.90 each,10 available,,,,,https://www.ebay.com.au/itm/194505420468?_ul=A...,https://vi.vipr.ebaydesc.com/itmdesc/194505420...,"{""market"": ""AU"", ""page"": 11, ""data"": {""scopedC...","{""0"": ""https://i.ebayimg.com/images/g/~04AAOSw...","{""0"": {""Condition"": ""Brand New: A brand-new, u..."
335,364,194505420622,Genuine Yaris SCP90 NSP90 NSP151 NSP150 Heater...,,AU $44.90 each,10 available,3 sold,,,,https://www.ebay.com.au/itm/194505420622?_ul=A...,https://vi.vipr.ebaydesc.com/itmdesc/194505420...,"{""market"": ""AU"", ""page"": 11, ""data"": {""scopedC...","{""0"": ""https://i.ebayimg.com/images/g/ieMAAOSw...","{""0"": {""Condition"": ""Brand New: A brand-new, u..."
336,365,194505420769,Genuine Yaris Vios NSP150 NCP151 NCP150 Heater...,,AU $44.90 each,10 available,1 sold,,,,https://www.ebay.com.au/itm/194505420769?_ul=A...,https://vi.vipr.ebaydesc.com/itmdesc/194505420...,"{""market"": ""AU"", ""page"": 11, ""data"": {""scopedC...","{""0"": ""https://i.ebayimg.com/images/g/T-wAAOSw...","{""0"": {""Condition"": ""Brand New: A brand-new, u..."


In [2]:
from tqdm import tqdm

output = pd.DataFrame()
for i in tqdm(range(len(input_)), desc='Progress', ncols=77):
    json_ = json.loads(input_.loc[i, 'Json_Compatibility'])
    if json_['market'] == 'US':
        list_vehicle_url = [f'''https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    elif json_['market'] == 'DE':
        list_vehicle_url = [f'''https://www.ebay.de/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    elif json_['market'] == 'UK':
        list_vehicle_url = [f'''https://www.ebay.co.uk/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    elif json_['market'] == 'AU':
        list_vehicle_url = [f'''https://www.ebay.com.au/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    df_temp = pd.DataFrame({'No': input_.loc[i, 'No'],
                            'Item Number': input_.loc[i, 'Item Number'],
                            'Market': json_['market'],
                            'Page': [j+1 for j in range(json_['page'])],
                            'JOIN_INP': [f'''{input_.loc[i, 'Item Number']};{j+1}''' for j in range(json_['page'])],
                            'Vehicle Url': list_vehicle_url,
                            'Data': json.dumps(json_['data'])})

    output = pd.concat([output, df_temp], ignore_index=True).fillna('')

output.to_excel(f'4.item_2.xlsx', index=False)

output

Progress: 100%|███████████████████████████| 338/338 [00:00<00:00, 512.73it/s]


,No,Item Number,Market,Page,JOIN_INP,Vehicle Url,Data
0,1,192511998212,AU,1,192511998212;1,https://www.ebay.com.au/g/api/finders?module_g...,"{""scopedContext"": {""catalogDetails"": {""categor..."
1,2,195660971887,AU,1,195660971887;1,https://www.ebay.com.au/g/api/finders?module_g...,"{""scopedContext"": {""catalogDetails"": {""categor..."
2,3,192557565418,AU,1,192557565418;1,https://www.ebay.com.au/g/api/finders?module_g...,"{""scopedContext"": {""catalogDetails"": {""categor..."
3,4,192335808616,AU,1,192335808616;1,https://www.ebay.com.au/g/api/finders?module_g...,"{""scopedContext"": {""catalogDetails"": {""categor..."
4,4,192335808616,AU,2,192335808616;2,https://www.ebay.com.au/g/api/finders?module_g...,"{""scopedContext"": {""catalogDetails"": {""categor..."
...,...,...,...,...,...,...,...
13165,366,194505420840,AU,7,194505420840;7,https://www.ebay.com.au/g/api/finders?module_g...,"{""scopedContext"": {""catalogDetails"": {""categor..."
13166,366,194505420840,AU,8,194505420840;8,https://www.ebay.com.au/g/api/finders?module_g...,"{""scopedContext"": {""catalogDetails"": {""categor..."
13167,366,194505420840,AU,9,194505420840;9,https://www.ebay.com.au/g/api/finders?module_g...,"{""scopedContext"": {""catalogDetails"": {""categor..."
13168,366,194505420840,AU,10,194505420840;10,https://www.ebay.com.au/g/api/finders?module_g...,"{""scopedContext"": {""catalogDetails"": {""categor..."
